# Building Fast Queries on a CSV

In this project, we will imagine that we own an online laptop store and want to build a way to answer a few different business questions about our inventory.

We will use the `laptops.csv` file as our inventory. This CSV file was adapted from the [Laptop Prices dataset on Kaggle](https://www.kaggle.com/ionaskel/laptop-prices). We changed the IDs and made the prices integers.

Here are the first few rows of the dataset:

In [1]:
import csv

with open('laptops.csv') as file:
    rows = list(csv.reader(file))
    header = rows[0]
    rows = rows[1:]
    print(header)
    print('\n')
    for row in rows[:5]:
        print(row)
        print('\n')

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']


['6571244', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', '1339']


['7287764', 'Apple', 'Macbook Air', 'Ultrabook', '13.3', '1440x900', 'Intel Core i5 1.8GHz', '8GB', '128GB Flash Storage', 'Intel HD Graphics 6000', 'macOS', '1.34kg', '898']


['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', '575']


['9722156', 'Apple', 'MacBook Pro', 'Ultrabook', '15.4', 'IPS Panel Retina Display 2880x1800', 'Intel Core i7 2.7GHz', '16GB', '512GB SSD', 'AMD Radeon Pro 455', 'macOS', '1.83kg', '2537']


['8550527', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 3.1GHz', '8GB', '2

Here is a brief description of the rows:

+ **ID**: A unique identifier for the laptop.
+ **Company**: The name of the company that produces the laptop.
+ **Product**: The name of the laptop.
+ **TypeName**: The type of laptop.
+ **Inches**: The size of the screen in inches.
+ **ScreenResolution**: The resolution of the screen.
+ **CPU**: The laptop CPU.
+ **RAM**: The amount of RAM in the laptop.
+ **Memory**: The size of the hard drive.
+ **GPU**: The graphics card name.
+ **OpSys**: The name of the operating system.
+ **Weight**: The laptop weight.
+ **Price**: The price of the laptop.

Let's explore the data and print the first few rows before we start implementing a way to represent this data as the store inventory.

## Inventory Class

The goal of this project is to create a class that represents our inventory. The methods in that class will implement the queries that we want to answer about our inventory. We will also preprocess that data to make those queries run faster.

Here are some queries that we will want to answer:

+ Given a laptop id, find the corresponding data.
+ Given an amount of money, find whether there are two laptops whose total price is that given amount.

Identify all laptops whose price falls within a given budget.
Let's start by implementing the constructor. It will take the name of the CSV file as argument and then read the rows contained in it.

In [2]:
class Inventory():
    
    def __init__(self, csv_filename):
        with open(csv_filename) as f: 
            reader = csv.reader(f)
            rows = list(csv.reader(f))
        self.header = rows[0]         
        self.rows = rows[1:]
        for row in self.rows:         
            row[-1] = int(row[-1])
            
inventory = Inventory('laptops.csv')
print(inventory.header)
print(len(inventory.rows))

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']
1303


## Finding a Laptop from the ID

The first thing that we will implement is a way to look up a laptop from a given identifier. In this way, when a customer comes to our store with a purchase slip, we can quickly identify the laptop to which it corresponds.

![](https://dq-content.s3.amazonaws.com/481/laptop_id_search.gif)

For this, we will write a function named `get_laptop_from_id()`. This function will take as argument the identifier of the laptop and return the full row of the laptop with that id.

In [3]:
class Inventory():
    def __init__(self, csv_filename):
        with open(csv_filename) as f: 
            reader = csv.reader(f)
            rows = list(csv.reader(f))
        self.header = rows[0]         
        self.rows = rows[1:]
        for row in self.rows:         
            row[-1] = int(row[-1])
            
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
            
inventory = Inventory('laptops.csv')
print(inventory.get_laptop_from_id('3362737'))
print(inventory.get_laptop_from_id('3362736'))

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
None


## Improving Id Lookups

This algorithm requires us to look at every single row to find the one that we are looking for (or decide that such a row does not exist). It has time complexity *O(R)* where *R* is the number of rows.

We can solve this problem more efficiently by preprocessing the data. Using a set, we can check in constant time whether a given identifier exists. However, we don't just want to know if it exists, we also want to retrieve the remaining row information. Therefore, we will use a dictionary instead of a set. Dictionaries have the same fast lookup properties that sets have, but allow us to associate values to the keys.

The idea is proceprocess the data into a dictionary where the keys are the IDs and the values the rows. Then, we will use that dictionary in the `get_laptop_from_id()` method. We can do this by:

1. Preprocess the data and create the dictionary in the `__init__()` method.
2. Re-implement the `get_laptop_from_id()` method. We will do it as a new method so that we can compare the two.

In [4]:
class Inventory():
    def __init__(self, csv_filename):
        with open(csv_filename) as f: 
            reader = csv.reader(f)
            rows = list(csv.reader(f))
        self.header = rows[0]         
        self.rows = rows[1:]
        for row in self.rows:         
            row[-1] = int(row[-1])
        self.id_to_row = {}   
        for row in self.rows:
            self.id_to_row[row[0]] = row
    
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
inventory = Inventory('laptops.csv')
print(inventory.get_laptop_from_id_fast('3362737'))
print(inventory.get_laptop_from_id_fast('3362736'))

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
None


## Comparing the Performance

The `get_laptop_from_id()` method has time complexity *O(R)* where *R* is the number of rows. In contrast, the new implementation as time complexity *O(1)*. It does so by using more memory to store the `self.id_to_row` dictionary and using a bit more time creating an instance (because it needs to create the dictionary).

Let's experiment to compare the performance of the two methods. The idea is to generate random IDs using the `random` module. Then, use both methods to lookup these same IDs. We will use the `time` module to measure the execution time of each lookup and, for each method, add all times together.

In [5]:
import time
import random 

ids = [str(random.randint(1000000, 9999999)) for _ in range(10000)]

inventory = Inventory('laptops.csv')

total_time_no_dict = 0

for each_id in ids:
    start = time.time()
    inventory.get_laptop_from_id(each_id)
    end = time.time()
    runtime = end - start
    total_time_no_dict += runtime

total_time_dict = 0

for each_id in ids:
    start = time.time()
    inventory.get_laptop_from_id_fast(each_id)
    end = time.time()
    runtime = end - start
    total_time_dict += runtime

print(total_time_no_dict)
print(total_time_dict)

1.1013686656951904
0.004996299743652344


The get_laptop_from_id_fast() method is significantly faster than the get_laptop_from_id() one. It is about 453 times faster!

## Two Laptop Promotion

Sometimes, the store offers a promotion where you give a gift card. A customer can use the gift to buy up to two laptops. To avoid having to keep track of what was already spent, the gift card has a single time usage. This means that, even if there is leftover money, it cannot be used anymore.

For example, imagine that there are only three laptops in inventory:

![](https://dq-content.s3.amazonaws.com/481/laptop_promotion.png)

The prices of these laptops are 1,339, 898, and 575 (in $ amounts). Say we offered a gift card of 2,500. Since a customer can buy, at most, two laptops with a gift card, the maximum they can spend is 2,237 (1,339 plus 898). Therefore, they might feel cheated because, no matter how they spend their gift card, they cannot spend the full 2,500.

Since we don't want to make a customer feel cheated, whenever we issue a gift card, we want to make sure that there is at least one way to spend it in full. In other words, before issuing a gift card for *D* dollars, we want to make sure that either there is a laptop that costs exactly *D* dollars or two laptops whose costs add up to precisely *D* dollars.

We write a function that, given a dollar amount, checks whether it is possible to spend precisely that amount by purchasing up to two laptops.

In [6]:
class Inventory():
    def __init__(self, csv_filename):
        with open(csv_filename) as f: 
            reader = csv.reader(f)
            rows = list(csv.reader(f))
        self.header = rows[0]         
        self.rows = rows[1:]
        for row in self.rows:         
            row[-1] = int(row[-1])
        self.id_to_row = {}   
        for row in self.rows:
            self.id_to_row[row[0]] = row
    
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if row[-1] == dollars:
                return True
        for row in rows:
            for row in rows:
                price_added = row[-1] + row[-1]
                if price_added == dollars:
                    return True
        return False
    
inventory = Inventory('laptops.csv')
print(inventory.check_promotion_dollars(1000))
print(inventory.check_promotion_dollars(442))

True
False


## Optimizing Laptop Promotion

Earlier, we preprocessed data to answer the kind of queries that we used in the `check_promotion_dollars()`. Let's implement this to make our code run faster.

Since we only care about whether or not there is a solution, we can store all laptops prices in a set when we initialize the inventory. Then we can check in constant time whether there is a laptop with a given price.

In [7]:
class Inventory():
    def __init__(self, csv_filename):
        with open(csv_filename) as f: 
            reader = csv.reader(f)
            rows = list(csv.reader(f))
        self.header = rows[0]         
        self.rows = rows[1:]
        for row in self.rows:         
            row[-1] = int(row[-1])
        self.id_to_row = {}   
        for row in self.rows:
            self.id_to_row[row[0]] = row
        self.prices = set()
        for row in self.rows:
            self.prices.add(row[-1])
    
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if row[-1] == dollars:
                return True
        for row in rows:
            for row in rows:
                price_added = row[-1] + row[-1]
                if price_added == dollars:
                    return True
        return False
    
    def check_promotion_dollars_fast(self, dollars):
        if self.prices == dollars:
            return True
        for price in self.prices:                   
            if dollars - price in self.prices:
                return True
        return False  
    
inventory = Inventory('laptops.csv')
print(inventory.check_promotion_dollars(1000))
print(inventory.check_promotion_dollars(442))

True
False


## Comparing Promotion Functions

Let's compare the performance of the last two functions that we wrote.

In [8]:
prices = [random.randint(100, 5000) for _ in range(100)]

inventory = Inventory('laptops.csv')

total_time_no_set = 0

for price in prices:
    start = time.time()
    inventory.check_promotion_dollars(price)
    end = time.time()
    runtime = end - start
    total_time_no_set += runtime

total_time_set = 0

for price in prices:
    start = time.time()
    inventory.check_promotion_dollars_fast(price)
    end = time.time()
    runtime = end - start
    total_time_set += runtime

print(total_time_no_set)
print(total_time_set)

28.8175528049469
0.0009996891021728516


We can see a significant improve in performance. The `check_promotion_dollars_fast()` method is 28826 times faster than the `check_promotion_dollars()` one.

## Finding Laptops Within a Budget

In this project, we are going to leverage and extend  a binary search algorithm to help a customer find all laptops that fall within their budget.

More formally, we want to write a method that efficiently answers the query: "Given a budget of *D* dollars, find all laptops whose price it at most *D*".

If we sort all laptops by price, we can use binary search to identify the first laptop in the sorted list with a price larger than *D*. We need to make sure that our binary search finds the *first one* on the list. Then, the result of the query will consist of all laptops whose index in the sorted list is smaller than the index of the first laptop whose price is higher than *D* dollars:

![](https://dq-content.s3.amazonaws.com/481/laptop_budget.png)

In [9]:
def row_price(row):
    return row[-1]

class Inventory():
    def __init__(self, csv_filename):
        with open(csv_filename) as f: 
            rows = list(csv.reader(f))
        self.header = rows[0]         
        self.rows = rows[1:]
        for row in self.rows:         
            row[-1] = int(row[-1])
        self.id_to_row = {}   
        for row in self.rows:
            self.id_to_row[row[0]] = row
        self.prices = set()
        for row in self.rows:
            self.prices.add(row[-1])
        self.rows_by_price = sorted(self.rows, key=row_price)
    
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if row[-1] == dollars:
                return True
        for row in rows:
            for row in rows:
                price_added = row[-1] + row[-1]
                if price_added == dollars:
                    return True
        return False
    
    def check_promotion_dollars_fast(self, dollars):
        if self.prices == dollars:
            return True
        for price in self.prices:                   
            if dollars - price in self.prices:
                return True
        return False  
    
    def find_first_laptop_more_expensive(self, target_price):
        range_start = 0                                   
        range_end = len(self.rows_by_price) - 1                       
        while range_start < range_end:
            range_middle = (range_end + range_start) // 2  
            price = self.rows_by_price[range_middle][-1]
            if price > target_price:                            
                range_end = range_middle                                   
            else:                                          
                range_start = range_middle + 1 
        price = self.rows_by_price[range_start][-1]
        if price <= target_price:                  
            return -1                                      
        return range_start
    
inventory = Inventory('laptops.csv')
print(inventory.find_first_laptop_more_expensive(1000))
print(inventory.find_first_laptop_more_expensive(10000))

683
-1
